### Relevant code : 

In [2]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import mediapipe as mp
import cv2
import torch

Function to get the pose from the frame

In [4]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, enable_segmentation=False, min_detection_confidence=0.5)

def extract_pose_data(frame):
    # Convert the color space from BGR to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame to detect the pose
    results = pose.process(frame_rgb)

    # Check if pose landmarks are detected
    if results.pose_landmarks:
        # Extract landmarks data
        pose_data = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]).flatten()
        return pose_data
    print("Failed to process, appening trash")
    return np.zeros(132)  # Assuming 33 landmarks each with 4 values (x, y, z, visibility)


I0000 00:00:1701177114.777131       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Generating the sequences from the dataset

In [ ]:
def create_image_sequences(image_folder, sequence_length):
    sequences = []
    image_files = sorted(os.listdir(image_folder))

    for i in tqdm(range(len(image_files) - sequence_length + 1), desc="Processing image sequences"):
        sequence = []

        for j in range(i, i + sequence_length):
            img_path = os.path.join(image_folder, image_files[j])
            frame = cv2.imread(img_path)
            print(frame)
            if frame is None:
                print(f"Failed to read image: {img_path}")
                break
            pose_data = extract_pose_data(frame)
            sequence.append(pose_data)

        if len(sequence) == sequence_length:
            sequences.append(sequence)

    return np.array(sequences)

def create_label_sequences(label_folder, sequence_length, image_files):
    labels = []

    for i in tqdm(range(len(image_files) - sequence_length + 1), desc="Processing label sequences"):
        sequence_labels = []

        for j in range(i, i + sequence_length):
            label_path = os.path.join(label_folder, image_files[j].replace('.png', '.txt'))
            with open(label_path, 'r') as file:
                if file is None:
                    print(f"failed to read label: {label_path}")
                fall_label = int(file.read().split()[0])
                sequence_labels.append(fall_label)

        # Use the most frequent label in the sequence as the sequence label
        if sequence_labels:
            labels.append(max(set(sequence_labels), key=sequence_labels.count))

    return np.array(labels)

# Example usage
sequence_length = 30
image_folder = "/Users/varunshankarhoskere/Desktop/Academics/PES/Capstone/Datasets/images"
label_folder = "/Users/varunshankarhoskere/Desktop/Academics/PES/Capstone/Datasets/labels"

image_sequences = create_image_sequences(image_folder, sequence_length)
image_files = sorted(os.listdir(image_folder))  # Ensure this matches the sorting in image sequence creation
label_sequences = create_label_sequences(label_folder, sequence_length, image_files)

# Now image_sequences and label_sequences contain your data


In [ ]:
import numpy as np

# Saving the image sequences
np.save('/path/to/save/image_sequences.npy', image_sequences)

# Saving the label sequences
np.save('/path/to/save/label_sequences.npy', label_sequences)

In [29]:
# Loading the image sequences
image_sequences = np.load('image_sequences.npy')

# Loading the label sequences
label_sequences = np.load('label_sequences.npy')

In [30]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [7]:
# Convert sequences and labels to PyTorch tensors
sequences_tensor = torch.tensor(image_sequences, dtype=torch.float32)
labels_tensor = torch.tensor(label_sequences, dtype=torch.long)

# Create a dataset and data loader
dataset = TensorDataset(sequences_tensor, labels_tensor)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# LSTM model initialization and training loop
# (Use the FallDetectionLSTM class and training loop provided earlier)


In [35]:
import torch.nn as nn

class FallDetectionLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(FallDetectionLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Initialize hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)


        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


In [37]:
from sklearn.model_selection import train_test_split

# Assuming image_sequences and label_sequences are your data
train_seqs, test_seqs, train_labels, test_labels = train_test_split(
    image_sequences, label_sequences, test_size=0.2, random_state=42
)

train_dataset = TensorDataset(torch.tensor(train_seqs, dtype=torch.float32), torch.tensor(train_labels, dtype=torch.long))
test_dataset = TensorDataset(torch.tensor(test_seqs, dtype=torch.float32), torch.tensor(test_labels, dtype=torch.long))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [38]:
import torch.optim as optim

In [39]:
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        for sequences, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

def evaluate_model(model, test_loader):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for sequences, labels in test_loader:
            outputs = model(sequences)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')

input_size = 132
hidden_size = 132
num_layers = 3
num_classes = 2

# Model Initialization
model = FallDetectionLSTM(input_size, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training and Evaluation
train_model(model, train_loader, criterion, optimizer, num_epochs=10)
evaluate_model(model, test_loader)


Epoch [1/10], Loss: 0.1040
Epoch [2/10], Loss: 0.1198
Epoch [3/10], Loss: 0.0223
Epoch [4/10], Loss: 0.0289
Epoch [5/10], Loss: 0.0752
Epoch [6/10], Loss: 0.0002
Epoch [7/10], Loss: 0.0026
Epoch [8/10], Loss: 0.0544
Epoch [9/10], Loss: 0.1417
Epoch [10/10], Loss: 0.0009
Accuracy: 98.03%


In [20]:
torch.save(model.state_dict(), 'lstm_model.pth')

In [ ]:
import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, enable_segmentation=False, min_detection_confidence=0.5)

def extract_pose_data(frame):
    # Process frame with MediaPipe Pose
    print(frame)
    results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    # Extract pose landmarks or other relevant data
    # This should match the format of your training data
    # ...
    if results.pose_landmarks:
        # Extract pose landmarks and flatten the array
        pose_data = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]).flatten()
        return pose_data
    return np.zeros(132)

cap = cv2.VideoCapture('/Users/varunshankarhoskere/Downloads/WhatsApp Video 2023-11-25 at 13.16.04.mp4')
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
sequence_length = 30  # Assuming this is the sequence length used during training

all_sequences = []
current_sequence = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    pose_data = extract_pose_data(frame)
    current_sequence.append(pose_data)

    if len(current_sequence) == sequence_length:
        all_sequences.append(current_sequence.copy())
        current_sequence.pop(0)  # Slide the window

cap.release()


In [16]:
# Assuming your model class is FallDetectionLSTM
# and you have a saved model file 'lstm_model.pth'
model = FallDetectionLSTM(input_size, hidden_size, num_layers, num_classes)
model.load_state_dict(torch.load('lstm_model.pth'))
model.eval()

# Convert sequences to tensor
sequences_tensor = torch.tensor(all_sequences, dtype=torch.float32)

# Make predictions
with torch.no_grad():
    for sequence in sequences_tensor:
        sequence = sequence.unsqueeze(0)  # Add batch dimension
        output = model(sequence)
        _, predicted = torch.max(output.data, 1)
        if predicted==1 : print(predicted)
        # Now `predicted` is your model's prediction for this sequence
        # ...


In [ ]:
model = FallDetectionLSTM(input_size, hidden_size, num_layers, num_classes)
model.load_state_dict(torch.load('lstm_model.pth'))
model.eval()

# Assuming sequences_tensor is your input data
with torch.no_grad():
    for sequence in sequences_tensor:
        sequence = sequence.unsqueeze(0)
        output = model(sequence)
        predicted_class = torch.sigmoid(output).round()  # For binary classification with sigmoid
        # or use torch.max for softmax
        print(predicted_class)


In [24]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, enable_segmentation=False, min_detection_confidence=0.5)

def extract_pose_data(frame):
    # Process frame with MediaPipe Pose
    print(frame)
    results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    # Extract pose landmarks or other relevant data
    # This should match the format of your training data
    # ...
    if results.pose_landmarks:
        # Extract pose landmarks and flatten the array
        pose_data = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]).flatten()
        return pose_data
    return np.zeros(132)

path = "/Users/varunshankarhoskere/Downloads/WA Test/output WhatsApp Video 2023-11-25 at 13/"

sequence = []

for image in os.listdir(path):
    frame = cv2.imread(path+image)
    print(frame)
    if frame is None:
        print(f"Failed to read image: {path}")
        break
    pose_data = extract_pose_data(frame)
    sequence.append(pose_data)

I0000 00:00:1701254861.913082       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro


[[[ 94 106 108]
  [ 93 105 107]
  [ 91 103 105]
  ...
  [199 188 174]
  [199 188 174]
  [199 188 174]]

 [[ 94 106 108]
  [ 92 104 106]
  [ 90 102 104]
  ...
  [198 187 173]
  [198 187 173]
  [198 187 173]]

 [[ 93 105 107]
  [ 91 103 105]
  [ 90 102 104]
  ...
  [195 184 170]
  [195 184 170]
  [195 184 170]]

 ...

 [[ 29  57  64]
  [ 31  59  66]
  [ 34  62  69]
  ...
  [ 81  97 104]
  [ 88 101 109]
  [114 125 133]]

 [[ 35  63  70]
  [ 33  61  68]
  [ 36  64  71]
  ...
  [ 84 100 107]
  [ 85  98 106]
  [ 93 104 112]]

 [[ 37  65  72]
  [ 32  60  67]
  [ 37  65  72]
  ...
  [ 91 107 114]
  [ 90 103 111]
  [ 81  92 100]]]
[[[ 94 106 108]
  [ 93 105 107]
  [ 91 103 105]
  ...
  [199 188 174]
  [199 188 174]
  [199 188 174]]

 [[ 94 106 108]
  [ 92 104 106]
  [ 90 102 104]
  ...
  [198 187 173]
  [198 187 173]
  [198 187 173]]

 [[ 93 105 107]
  [ 91 103 105]
  [ 90 102 104]
  ...
  [195 184 170]
  [195 184 170]
  [195 184 170]]

 ...

 [[ 29  57  64]
  [ 31  59  66]
  [ 34  62  69]
  ..

In [25]:
input_size = 132
hidden_size = 132
num_layers = 3
num_classes = 2

In [26]:
model = FallDetectionLSTM(input_size, hidden_size, num_layers, num_classes)
model.load_state_dict(torch.load('lstm_model.pth'))
model.eval()

sequences_tensor = torch.tensor(sequence, dtype=torch.float32)



# Make predictions
with torch.no_grad():
    for sequence in sequences_tensor:
        sequence = sequence.unsqueeze(0)  # Add batch dimension
        output, (hn, cn) = model(sequence, (h0, c0))  # Pass initial states to LSTM
        _, predicted = torch.max(output.data, 1)
        print(predicted)


TypeError: FallDetectionLSTM.forward() takes 2 positional arguments but 3 were given

In [22]:

model = FallDetectionLSTM(input_size, hidden_size, num_layers, num_classes)
model.load_state_dict(torch.load('lstm_model.pth'))
model.eval()

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, enable_segmentation=False, min_detection_confidence=0.5)

def extract_pose_data(frame):
    # Process frame with MediaPipe Pose
    print(frame)
    results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    # Extract pose landmarks or other relevant data
    # This should match the format of your training data
    # ...
    if results.pose_landmarks:
        # Extract pose landmarks and flatten the array
        pose_data = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]).flatten()
        return pose_data
    return np.zeros(132)

path = "/Users/varunshankarhoskere/Downloads/WA Test/output WhatsApp Video 2023-11-25 at 13/"

sequence = []

for image in os.listdir(path):
    frame = cv2.imread(path+image)
    print(frame)
    if frame is None:
        print(f"Failed to read image: {path}")
        break
    pose_data = extract_pose_data(frame)
    # sequence.append(pose_data)
    # sequences_tensor = torch.tensor(sequence, dtype=torch.float32)
    sequences_tensor = torch.tensor(pose_data, dtype=torch.float32)
    with torch.no_grad():
        # sequence = sequence.unsqueeze(0)  # Add batch dimension
        output = model(torch.tensor(sequences_tensor.unsqueeze(0)))
        _, predicted = torch.max(output.data, 1)
        print(predicted)

# Make predictions


    

I0000 00:00:1701254750.599389       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro
/var/folders/6r/rq8wfq31581cybh61k1_sz8m0000gn/T/ipykernel_23069/2418193738.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = model(torch.tensor(sequences_tensor.unsqueeze(0)))


[[[ 94 106 108]
  [ 93 105 107]
  [ 91 103 105]
  ...
  [199 188 174]
  [199 188 174]
  [199 188 174]]

 [[ 94 106 108]
  [ 92 104 106]
  [ 90 102 104]
  ...
  [198 187 173]
  [198 187 173]
  [198 187 173]]

 [[ 93 105 107]
  [ 91 103 105]
  [ 90 102 104]
  ...
  [195 184 170]
  [195 184 170]
  [195 184 170]]

 ...

 [[ 29  57  64]
  [ 31  59  66]
  [ 34  62  69]
  ...
  [ 81  97 104]
  [ 88 101 109]
  [114 125 133]]

 [[ 35  63  70]
  [ 33  61  68]
  [ 36  64  71]
  ...
  [ 84 100 107]
  [ 85  98 106]
  [ 93 104 112]]

 [[ 37  65  72]
  [ 32  60  67]
  [ 37  65  72]
  ...
  [ 91 107 114]
  [ 90 103 111]
  [ 81  92 100]]]
[[[ 94 106 108]
  [ 93 105 107]
  [ 91 103 105]
  ...
  [199 188 174]
  [199 188 174]
  [199 188 174]]

 [[ 94 106 108]
  [ 92 104 106]
  [ 90 102 104]
  ...
  [198 187 173]
  [198 187 173]
  [198 187 173]]

 [[ 93 105 107]
  [ 91 103 105]
  [ 90 102 104]
  ...
  [195 184 170]
  [195 184 170]
  [195 184 170]]

 ...

 [[ 29  57  64]
  [ 31  59  66]
  [ 34  62  69]
  ..

RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors

In [4]:
import cv2
import mediapipe as mp
import os
import time

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Folder containing the image files
image_folder = '/Users/varunshankarhoskere/Downloads/WA Test/output WhatsApp Video 2023-11-25 at 13/'

# List all image files in the folder
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg'))]

# Process each image file
for image_file in image_files:
    # Read the image
    image_path = os.path.join(image_folder, image_file)
    frame = cv2.imread(image_path)

    # Process the image for pose detection
    results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Draw the pose annotations on the image
    if results.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Display the image
    cv2.imshow('Pose', frame)

    # Wait for a key press or display for a set time
    if cv2.waitKey(1000) & 0xFF == 27:  # Display each frame for 1000 milliseconds
        break

    time.sleep(1)  # Add delay to see each image for a second

cv2.destroyAllWindows()


I0000 00:00:1701251701.844887       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1 Pro


KeyboardInterrupt: 